In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imutils import paths
import cv2
from PIL import Image
#from mtcnn import MTCNN
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from random import choice
#import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model

In [7]:
# load face detector model 
detector= cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [8]:
# load samples
path1 = list(paths.list_images(os.path.join(os.getcwd(), 'jeska2')))
path2 = list(paths.list_images(os.path.join(os.getcwd(), 'hue2')))
path3 = list(paths.list_images(os.path.join(os.getcwd(), 'mina2')))
path4 = list(paths.list_images(os.path.join(os.getcwd(), 'chu2')))
path5 = list(paths.list_images(os.path.join(os.getcwd(), 'nga2')))
path6 = list(paths.list_images(os.path.join(os.getcwd(), 'kha2')))

In [9]:
paths = [path1, path2, path3, path4, path5, path6]

In [15]:
# preprocess images
face_samples = []
fs_len = []

for path in paths:
    length = 0
    for i in range(len(path)):
        # load the image and converting it to gray scale
        pilImage = Image.open(path[i]).rotate(-90)
        # convert the PIL image into numpy array
        imageNp = np.array(pilImage, 'uint8')
        # extract the face from the training image sample
        faces = detector.detectMultiScale(imageNp, scaleFactor=1.5, minNeighbors=5)
        # resize faces to 160x160, covert them to array and append them to the list
        for (x,y,w,h) in faces:
            face = imageNp[y:y+h,x:x+w]
            image = Image.fromarray(face)
            image = image.resize((160,160))
            face_array = np.asarray(image)
            face_samples.append(face_array)
            length += 1

    fs_len.append(length)
    
print(len(face_samples))
print(fs_len)

469
[80, 74, 85, 79, 79, 72]


In [16]:
# labels vector
labels = np.concatenate((np.zeros((80, 1)), np.ones((74, 1)), np.ones((85, 1))*2, np.ones((79, 1))*3, np.ones((79, 1))*4, np.ones((72, 1))*5), axis = 0)
print(labels.shape)

(469, 1)


In [17]:
labels = labels.astype(int)

In [18]:
# load Facenet model
facenet = load_model('facenet_keras.h5')

In [19]:
# use Facenet model to create embedding vector to each sample

embeddings = []

for fs in face_samples:
    fs = fs.astype('float32')
    mean, std = fs.mean(), fs.std()
    fs = (fs - mean) / std
    sample = np.expand_dims(fs, axis = 0)
    embedding = facenet.predict(sample)
    embeddings.append(embedding)

In [20]:
embeddings = np.asarray(embeddings)

In [24]:
embeddings = embeddings.reshape(469, 128)

In [25]:
# use Support Vector Machine algorithsm to classify the embedding vectors with corresponding labels
model = SVC(kernel = 'linear')
model.fit(embeddings, labels)

C:\Users\My Laptop\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [27]:
from joblib import dump, load

In [28]:
# save model
dump(model, 'facenet_svm.joblib') 

['facenet_svm.joblib']